In [138]:
import numpy as np

In [139]:
import pandas as pd

In [140]:
datas=['bfly','bfnx','bnus','btrx','cbse','gmni','itbi','krkn','lmax','okcn','stmp'] #Importe les données
exclist=[] #Importe les données
for i in datas:
    df_temp=pd.read_csv('./data/'+i+'.csv')
    df_temp['exchange']=i
    exclist.append(df_temp)
    
bigdf=pd.concat(exclist)

In [141]:
bigdf.head()

,timestamp,price,amount,exchange
0,2021-02-24 23:59:33.720,49712.15,0.0054,bfly
1,2021-02-24 23:44:16.980,49603.44,0.0770,bfly
2,2021-02-24 22:32:39.320,48864.19,0.1200,bfly
3,2021-02-24 22:15:19.030,48252.47,0.1200,bfly
4,2021-02-24 22:15:17.157,48247.38,0.1200,bfly


In [142]:
bigdf['timestamp']=pd.to_datetime(bigdf['timestamp'])  #Modification et vérification de l'horodatage en tant que datetime    

In [143]:
bigdf.index=bigdf.timestamp
bigdf.drop(columns=['timestamp'],axis=1,inplace=True)
bigdf.head()

,price,amount,exchange
timestamp,,,
2021-02-24 23:59:33.720,49712.15,0.0054,bfly
2021-02-24 23:44:16.980,49603.44,0.0770,bfly
2021-02-24 22:32:39.320,48864.19,0.1200,bfly
2021-02-24 22:15:19.030,48252.47,0.1200,bfly
2021-02-24 22:15:17.157,48247.38,0.1200,bfly


In [144]:
bigdf.sort_values(by='exchange') #trier par noms d'échange
bfly_df=bigdf.groupby("exchange").get_group('bfly') #Processus de regroupement en fonction de chaque nom d'échange
bfly_df

,price,amount,exchange
timestamp,,,
2021-02-24 23:59:33.720,49712.15,0.005400,bfly
2021-02-24 23:44:16.980,49603.44,0.077000,bfly
2021-02-24 22:32:39.320,48864.19,0.120000,bfly
2021-02-24 22:15:19.030,48252.47,0.120000,bfly
2021-02-24 22:15:17.157,48247.38,0.120000,bfly
...,...,...,...
2021-02-24 00:14:48.747,48263.33,0.250300,bfly
2021-02-24 00:11:03.583,48574.05,0.000245,bfly
2021-02-24 00:11:02.263,48574.05,0.001000,bfly


In [145]:
bfly_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 234 entries, 2021-02-24 23:59:33.720000 to 2021-02-24 00:02:59.720000
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     234 non-null    float64
 1   amount    234 non-null    float64
 2   exchange  234 non-null    object 
dtypes: float64(2), object(1)
memory usage: 7.3+ KB


In [146]:
print(bfly_df.isnull().sum()) #Trouver les valeurs manquantes

price       0
amount      0
exchange    0
dtype: int64


In [147]:
def vwap(bfly_df):
    ap=sum(bfly_df.price.values*bfly_df.amount.values)
    a=sum(bfly_df.amount.values)
    if(a==0): return float('nan')
    return ap/a
bfly_vwap=bfly_df.groupby(pd.Grouper(freq="h")).apply(vwap)
bfly_vwap = bfly_vwap.astype(float)

In [148]:
bfly_vwap

timestamp
2021-02-24 00:00:00    48006.990396
2021-02-24 01:00:00    49607.923645
2021-02-24 02:00:00    50196.677303
2021-02-24 03:00:00    50948.000000
2021-02-24 04:00:00    50892.806229
2021-02-24 05:00:00             NaN
2021-02-24 06:00:00    49978.511632
2021-02-24 07:00:00    49680.100000
2021-02-24 08:00:00    50144.205464
2021-02-24 09:00:00    50884.023714
2021-02-24 10:00:00    50536.602041
2021-02-24 11:00:00    50474.184698
2021-02-24 12:00:00    51228.616364
2021-02-24 13:00:00    49681.734856
2021-02-24 14:00:00    49266.087850
2021-02-24 15:00:00    48785.145928
2021-02-24 16:00:00    49645.483600
2021-02-24 17:00:00    49714.249704
2021-02-24 18:00:00    49602.439024
2021-02-24 19:00:00    49191.600563
2021-02-24 20:00:00    48991.421225
2021-02-24 21:00:00    48404.532367
2021-02-24 22:00:00    48270.598647
2021-02-24 23:00:00    49610.564199
Freq: H, dtype: float64

In [149]:
bfly_vwap.fillna(bfly_vwap.mean(),inplace=True) #attribuer des valeurs moyennes pour remplacer les valeurs manquantes
print(bfly_vwap)

timestamp
2021-02-24 00:00:00    48006.990396
2021-02-24 01:00:00    49607.923645
2021-02-24 02:00:00    50196.677303
2021-02-24 03:00:00    50948.000000
2021-02-24 04:00:00    50892.806229
2021-02-24 05:00:00    49727.934759
2021-02-24 06:00:00    49978.511632
2021-02-24 07:00:00    49680.100000
2021-02-24 08:00:00    50144.205464
2021-02-24 09:00:00    50884.023714
2021-02-24 10:00:00    50536.602041
2021-02-24 11:00:00    50474.184698
2021-02-24 12:00:00    51228.616364
2021-02-24 13:00:00    49681.734856
2021-02-24 14:00:00    49266.087850
2021-02-24 15:00:00    48785.145928
2021-02-24 16:00:00    49645.483600
2021-02-24 17:00:00    49714.249704
2021-02-24 18:00:00    49602.439024
2021-02-24 19:00:00    49191.600563
2021-02-24 20:00:00    48991.421225
2021-02-24 21:00:00    48404.532367
2021-02-24 22:00:00    48270.598647
2021-02-24 23:00:00    49610.564199
Freq: H, dtype: float64


In [150]:
bfnx_df=bigdf.groupby("exchange").get_group('bfnx')
bfnx_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 318667 entries, 2021-02-24 23:59:58.181000 to 2021-02-24 00:00:00.516000
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   price     318667 non-null  float64
 1   amount    318667 non-null  float64
 2   exchange  318667 non-null  object 
dtypes: float64(2), object(1)
memory usage: 9.7+ MB


In [151]:
def vwap(bfnx_df):
    ap=sum(bfnx_df.price.values*bfnx_df.amount.values)
    a=sum(bfnx_df.amount.values)
    if(a==0): return float('nan')
    return ap/a
bfnx_vwap=bfnx_df.groupby(pd.Grouper(freq="h")).apply(vwap)
bfnx_vwap = bfnx_vwap.astype(float)
bfnx_vwap

timestamp
2021-02-24 00:00:00    47778.970482
2021-02-24 01:00:00    49441.592938
2021-02-24 02:00:00    50237.005036
2021-02-24 03:00:00    50608.443325
2021-02-24 04:00:00    50889.743300
2021-02-24 05:00:00    50344.152274
2021-02-24 06:00:00    49713.514206
2021-02-24 07:00:00    49899.138736
2021-02-24 08:00:00    50629.718533
2021-02-24 09:00:00    50887.439443
2021-02-24 10:00:00    50480.384254
2021-02-24 11:00:00    50494.052136
2021-02-24 12:00:00    50976.883072
2021-02-24 13:00:00    49701.256411
2021-02-24 14:00:00    49233.188583
2021-02-24 15:00:00    49011.131584
2021-02-24 16:00:00    49734.684912
2021-02-24 17:00:00    49788.338504
2021-02-24 18:00:00    49795.622560
2021-02-24 19:00:00    49214.864132
2021-02-24 20:00:00    48895.911947
2021-02-24 21:00:00    48440.911774
2021-02-24 22:00:00    48531.103989
2021-02-24 23:00:00    49385.749440
Freq: H, dtype: float64

In [152]:
bnus_df=bigdf.groupby("exchange").get_group('bnus')
bnus_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 53672 entries, 2021-02-24 23:59:58.164000 to 2021-02-24 00:00:00.790000
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     53672 non-null  float64
 1   amount    53672 non-null  float64
 2   exchange  53672 non-null  object 
dtypes: float64(2), object(1)
memory usage: 1.6+ MB


In [153]:
def vwap(bnus_df):
    ap=sum(bnus_df.price.values*bnus_df.amount.values)
    a=sum(bnus_df.amount.values)
    if(a==0): return float('nan')
    return ap/a
bnus_vwap=bnus_df.groupby(pd.Grouper(freq="h")).apply(vwap)
bnus_vwap = bnus_vwap.astype(float)
bnus_vwap

timestamp
2021-02-24 00:00:00    47846.763010
2021-02-24 01:00:00    49196.187346
2021-02-24 02:00:00    50200.414314
2021-02-24 03:00:00    50523.291435
2021-02-24 04:00:00    50900.796481
2021-02-24 05:00:00    50383.987434
2021-02-24 06:00:00    50046.486487
2021-02-24 07:00:00    49861.581152
2021-02-24 08:00:00    50559.886413
2021-02-24 09:00:00    50842.080628
2021-02-24 10:00:00    50515.867452
2021-02-24 11:00:00    50475.338733
2021-02-24 12:00:00    50870.863408
2021-02-24 13:00:00    49783.077635
2021-02-24 14:00:00    49251.723838
2021-02-24 15:00:00    49258.306863
2021-02-24 16:00:00    49765.878311
2021-02-24 17:00:00    49808.544722
2021-02-24 18:00:00    49795.325779
2021-02-24 19:00:00    49355.109623
2021-02-24 20:00:00    48906.476335
2021-02-24 21:00:00    48485.980606
2021-02-24 22:00:00    48549.925443
2021-02-24 23:00:00    49399.630134
Freq: H, dtype: float64

In [154]:
btrx_df=bigdf.groupby("exchange").get_group('btrx')
btrx_df.info()

def vwap(btrx_df):
    ap=sum(btrx_df.price.values*btrx_df.amount.values)
    a=sum(btrx_df.amount.values)
    if(a==0): return float('nan')
    return ap/a
btrx_vwap=btrx_df.groupby(pd.Grouper(freq="h")).apply(vwap)
btrx_vwap = btrx_vwap.astype(float)
btrx_vwap



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26789 entries, 2021-02-24 23:59:48.910000 to 2021-02-24 00:00:00.150000
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     26789 non-null  float64
 1   amount    26789 non-null  float64
 2   exchange  26789 non-null  object 
dtypes: float64(2), object(1)
memory usage: 837.2+ KB


timestamp
2021-02-24 00:00:00    47861.252787
2021-02-24 01:00:00    49343.265805
2021-02-24 02:00:00    50264.128120
2021-02-24 03:00:00    50528.684300
2021-02-24 04:00:00    50914.284328
2021-02-24 05:00:00    50537.358940
2021-02-24 06:00:00    50033.771901
2021-02-24 07:00:00    50032.963471
2021-02-24 08:00:00    50587.218865
2021-02-24 09:00:00    50947.529893
2021-02-24 10:00:00    50665.392921
2021-02-24 11:00:00    50556.416640
2021-02-24 12:00:00    50907.729530
2021-02-24 13:00:00    49839.742131
2021-02-24 14:00:00    49311.928912
2021-02-24 15:00:00    49280.555708
2021-02-24 16:00:00    49805.936932
2021-02-24 17:00:00    49786.375155
2021-02-24 18:00:00    49797.741514
2021-02-24 19:00:00    49257.819346
2021-02-24 20:00:00    48907.006080
2021-02-24 21:00:00    48486.392268
2021-02-24 22:00:00    48566.458228
2021-02-24 23:00:00    49488.224862
Freq: H, dtype: float64

In [155]:
cbse_df=bigdf.groupby("exchange").get_group('cbse')
cbse_df.info()

def vwap(cbse_df):
    ap=sum(cbse_df.price.values*cbse_df.amount.values)
    a=sum(cbse_df.amount.values)
    if(a==0): return float('nan')
    return ap/a
cbse_vwap=cbse_df.groupby(pd.Grouper(freq="h")).apply(vwap)
cbse_vwap = cbse_vwap.astype(float)
cbse_vwap

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 489110 entries, 2021-02-24 23:59:59.873000 to 2021-02-24 00:00:00.010000
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   price     489110 non-null  float64
 1   amount    489110 non-null  float64
 2   exchange  489110 non-null  object 
dtypes: float64(2), object(1)
memory usage: 14.9+ MB


timestamp
2021-02-24 00:00:00    47945.341624
2021-02-24 01:00:00    49317.201370
2021-02-24 02:00:00    50248.894775
2021-02-24 03:00:00    50498.142558
2021-02-24 04:00:00    50942.646520
2021-02-24 05:00:00    50513.989242
2021-02-24 06:00:00    49982.465002
2021-02-24 07:00:00    49959.624169
2021-02-24 08:00:00    50541.090694
2021-02-24 09:00:00    50901.001611
2021-02-24 10:00:00    50523.886721
2021-02-24 11:00:00    50498.455216
2021-02-24 12:00:00    50935.790979
2021-02-24 13:00:00    49811.472119
2021-02-24 14:00:00    49293.346463
2021-02-24 15:00:00    49224.856517
2021-02-24 16:00:00    49748.948299
2021-02-24 17:00:00    49813.534998
2021-02-24 18:00:00    49790.606855
2021-02-24 19:00:00    49259.860732
2021-02-24 20:00:00    48922.873467
2021-02-24 21:00:00    48476.878211
2021-02-24 22:00:00    48579.180689
2021-02-24 23:00:00    49428.972903
Freq: H, dtype: float64

In [156]:
gmni_df=bigdf.groupby("exchange").get_group('gmni')
gmni_df.info()

def vwap(gmni_df):
    ap=sum(gmni_df.price.values*gmni_df.amount.values)
    a=sum(gmni_df.amount.values)
    if(a==0): return float('nan')
    return ap/a
gmni_vwap=gmni_df.groupby(pd.Grouper(freq="h")).apply(vwap)
gmni_vwap = gmni_vwap.astype(float)
gmni_vwap

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 49329 entries, 2021-02-24 23:59:53.406000 to 2021-02-24 00:00:01.673000
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     49329 non-null  float64
 1   amount    49329 non-null  float64
 2   exchange  49329 non-null  object 
dtypes: float64(2), object(1)
memory usage: 1.5+ MB


timestamp
2021-02-24 00:00:00    47981.711507
2021-02-24 01:00:00    49458.079101
2021-02-24 02:00:00    50268.259968
2021-02-24 03:00:00    50494.398639
2021-02-24 04:00:00    50934.953286
2021-02-24 05:00:00    50528.602570
2021-02-24 06:00:00    49935.135891
2021-02-24 07:00:00    50034.858944
2021-02-24 08:00:00    50476.882959
2021-02-24 09:00:00    50901.639070
2021-02-24 10:00:00    50494.324252
2021-02-24 11:00:00    50446.875139
2021-02-24 12:00:00    50932.913060
2021-02-24 13:00:00    49750.219364
2021-02-24 14:00:00    49282.042100
2021-02-24 15:00:00    49249.513262
2021-02-24 16:00:00    49720.094217
2021-02-24 17:00:00    49843.673337
2021-02-24 18:00:00    49796.550879
2021-02-24 19:00:00    49204.725466
2021-02-24 20:00:00    48952.988324
2021-02-24 21:00:00    48526.697204
2021-02-24 22:00:00    48650.565402
2021-02-24 23:00:00    49457.500518
Freq: H, dtype: float64

In [157]:
itbi_df=bigdf.groupby("exchange").get_group('itbi')
itbi_df.info()

def vwap(itbi_df):
    ap=sum(itbi_df.price.values*itbi_df.amount.values)
    a=sum(itbi_df.amount.values)
    if(a==0): return float('nan')
    return ap/a
itbi_vwap=itbi_df.groupby(pd.Grouper(freq="h")).apply(vwap)
itbi_vwap = itbi_vwap.astype(float)
itbi_vwap

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 30187 entries, 2021-02-24 23:59:48.157000 to 2021-02-24 00:00:03.650000
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     30187 non-null  float64
 1   amount    30187 non-null  float64
 2   exchange  30187 non-null  object 
dtypes: float64(2), object(1)
memory usage: 943.3+ KB


timestamp
2021-02-24 00:00:00    47772.255562
2021-02-24 01:00:00    49509.699420
2021-02-24 02:00:00    50260.294897
2021-02-24 03:00:00    50533.802903
2021-02-24 04:00:00    51019.753640
2021-02-24 05:00:00    50416.281591
2021-02-24 06:00:00    49907.747367
2021-02-24 07:00:00    49944.963422
2021-02-24 08:00:00    50600.265267
2021-02-24 09:00:00    51014.427564
2021-02-24 10:00:00    50588.054390
2021-02-24 11:00:00    50500.795390
2021-02-24 12:00:00    50956.067366
2021-02-24 13:00:00    49790.028432
2021-02-24 14:00:00    49260.066525
2021-02-24 15:00:00    49062.979123
2021-02-24 16:00:00    49749.262596
2021-02-24 17:00:00    49815.871953
2021-02-24 18:00:00    49791.772547
2021-02-24 19:00:00    49411.327848
2021-02-24 20:00:00    48880.346896
2021-02-24 21:00:00    48526.432824
2021-02-24 22:00:00    48497.261357
2021-02-24 23:00:00    49493.097791
Freq: H, dtype: float64

In [158]:
krkn_df=bigdf.groupby("exchange").get_group('krkn')
krkn_df.info()

def vwap(krkn_df):
    ap=sum(krkn_df.price.values*krkn_df.amount.values)
    a=sum(krkn_df.amount.values)
    if(a==0): return float('nan')
    return ap/a
krkn_vwap=krkn_df.groupby(pd.Grouper(freq="h")).apply(vwap)
krkn_vwap = krkn_vwap.astype(float)
krkn_vwap

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75090 entries, 2021-02-24 23:59:58.649000 to 2021-02-24 00:00:01.115000
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     75090 non-null  float64
 1   amount    75090 non-null  float64
 2   exchange  75090 non-null  object 
dtypes: float64(2), object(1)
memory usage: 2.3+ MB


timestamp
2021-02-24 00:00:00    47698.307100
2021-02-24 01:00:00    49179.026877
2021-02-24 02:00:00    50214.502350
2021-02-24 03:00:00    50537.044841
2021-02-24 04:00:00    50917.060833
2021-02-24 05:00:00    50421.092041
2021-02-24 06:00:00    49916.108053
2021-02-24 07:00:00    49994.271641
2021-02-24 08:00:00    50545.216768
2021-02-24 09:00:00    50987.736822
2021-02-24 10:00:00    50643.536208
2021-02-24 11:00:00    50523.501578
2021-02-24 12:00:00    50864.097845
2021-02-24 13:00:00    49743.211946
2021-02-24 14:00:00    49223.085281
2021-02-24 15:00:00    49217.787308
2021-02-24 16:00:00    49749.960860
2021-02-24 17:00:00    49842.350098
2021-02-24 18:00:00    49813.098122
2021-02-24 19:00:00    49230.388097
2021-02-24 20:00:00    48964.871226
2021-02-24 21:00:00    48499.380608
2021-02-24 22:00:00    48540.346388
2021-02-24 23:00:00    49459.905292
Freq: H, dtype: float64

In [159]:
lmax_df=bigdf.groupby("exchange").get_group('lmax')
lmax_df.info()

def vwap(lmax_df):
    ap=sum(lmax_df.price.values*lmax_df.amount.values)
    a=sum(lmax_df.amount.values)
    if(a==0): return float('nan')
    return ap/a
lmax_vwap=lmax_df.groupby(pd.Grouper(freq="h")).apply(vwap)
lmax_vwap = lmax_vwap.astype(float)
lmax_vwap

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 27972 entries, 2021-02-24 23:59:59.691000 to 2021-02-24 00:00:00.650000
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     27972 non-null  float64
 1   amount    27972 non-null  float64
 2   exchange  27972 non-null  object 
dtypes: float64(2), object(1)
memory usage: 874.1+ KB


timestamp
2021-02-24 00:00:00    47880.958914
2021-02-24 01:00:00    49250.834067
2021-02-24 02:00:00    50227.852912
2021-02-24 03:00:00    50458.462794
2021-02-24 04:00:00    50851.638004
2021-02-24 05:00:00    50473.959563
2021-02-24 06:00:00    50061.808328
2021-02-24 07:00:00    49996.280196
2021-02-24 08:00:00    50552.860398
2021-02-24 09:00:00    50934.152165
2021-02-24 10:00:00    50514.769692
2021-02-24 11:00:00    50505.688077
2021-02-24 12:00:00    50918.641498
2021-02-24 13:00:00    49827.035500
2021-02-24 14:00:00    49274.408306
2021-02-24 15:00:00    49324.913761
2021-02-24 16:00:00    49755.518204
2021-02-24 17:00:00    49845.112441
2021-02-24 18:00:00    49796.547791
2021-02-24 19:00:00    49317.071732
2021-02-24 20:00:00    48952.608092
2021-02-24 21:00:00    48518.462038
2021-02-24 22:00:00    48645.530586
2021-02-24 23:00:00    49423.471087
Freq: H, dtype: float64

In [160]:
okcn_df=bigdf.groupby("exchange").get_group('okcn')
okcn_df.info()

def vwap(okcn_df):
    ap=sum(okcn_df.price.values*okcn_df.amount.values)
    a=sum(okcn_df.amount.values)
    if(a==0): return float('nan')
    return ap/a
okcn_vwap=okcn_df.groupby(pd.Grouper(freq="h")).apply(vwap)
okcn_vwap = okcn_vwap.astype(float)
okcn_vwap

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12459 entries, 2021-02-24 23:59:57.847000 to 2021-02-24 00:00:01.314000
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     12459 non-null  float64
 1   amount    12459 non-null  float64
 2   exchange  12459 non-null  object 
dtypes: float64(2), object(1)
memory usage: 389.3+ KB


timestamp
2021-02-24 00:00:00    47954.733900
2021-02-24 01:00:00    49422.816196
2021-02-24 02:00:00    50218.786802
2021-02-24 03:00:00    50435.243773
2021-02-24 04:00:00    51023.204299
2021-02-24 05:00:00    50399.457611
2021-02-24 06:00:00    49940.207735
2021-02-24 07:00:00    50030.400520
2021-02-24 08:00:00    50482.659430
2021-02-24 09:00:00    51005.570586
2021-02-24 10:00:00    50564.539463
2021-02-24 11:00:00    50556.759305
2021-02-24 12:00:00    50921.934715
2021-02-24 13:00:00    49953.170583
2021-02-24 14:00:00    49345.871820
2021-02-24 15:00:00    49288.106394
2021-02-24 16:00:00    49824.679690
2021-02-24 17:00:00    49817.802710
2021-02-24 18:00:00    49797.709996
2021-02-24 19:00:00    49343.114456
2021-02-24 20:00:00    48851.209268
2021-02-24 21:00:00    48451.587572
2021-02-24 22:00:00    48543.495588
2021-02-24 23:00:00    49568.860668
Freq: H, dtype: float64

In [161]:
stmp_df=bigdf.groupby("exchange").get_group('stmp')
stmp_df.info()

def vwap(stmp_df):
    ap=sum(stmp_df.price.values*stmp_df.amount.values)
    a=sum(stmp_df.amount.values)
    if(a==0): return float('nan')
    return ap/a
stmp_vwap=stmp_df.groupby(pd.Grouper(freq="h")).apply(vwap)
stmp_vwap = stmp_vwap.astype(float)
stmp_vwap

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 53284 entries, 2021-02-24 23:59:54 to 2021-02-24 00:00:35
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     53284 non-null  float64
 1   amount    53284 non-null  float64
 2   exchange  53284 non-null  object 
dtypes: float64(2), object(1)
memory usage: 1.6+ MB


timestamp
2021-02-24 00:00:00    47944.262770
2021-02-24 01:00:00    49220.299732
2021-02-24 02:00:00    50283.555918
2021-02-24 03:00:00    50472.637817
2021-02-24 04:00:00    50987.941708
2021-02-24 05:00:00    50440.124897
2021-02-24 06:00:00    49939.781897
2021-02-24 07:00:00    49974.235438
2021-02-24 08:00:00    50594.394449
2021-02-24 09:00:00    50902.708746
2021-02-24 10:00:00    50590.393464
2021-02-24 11:00:00    50480.338478
2021-02-24 12:00:00    50947.570599
2021-02-24 13:00:00    49893.768540
2021-02-24 14:00:00    49236.200293
2021-02-24 15:00:00    49221.752536
2021-02-24 16:00:00    49757.699345
2021-02-24 17:00:00    49847.050543
2021-02-24 18:00:00    49838.075835
2021-02-24 19:00:00    49337.141714
2021-02-24 20:00:00    48927.907365
2021-02-24 21:00:00    48509.784282
2021-02-24 22:00:00    48580.633397
2021-02-24 23:00:00    49504.553952
Freq: H, dtype: float64

In [162]:
vwap_df=pd.concat([bfly_vwap,bfnx_vwap,bnus_vwap,btrx_vwap,gmni_vwap,itbi_vwap,krkn_vwap,lmax_vwap,okcn_vwap,stmp_vwap],axis=1,)
vwap_df.columns=['bfly_vwap','bfnx_vwap','bnus_vwap','btrx_vwap','gmni_vwap','itbi_vwap','krkn_vwap','lmax_vwap','okcn_vwap','stmp_vwap']
vwap_df

,bfly_vwap,bfnx_vwap,bnus_vwap,btrx_vwap,gmni_vwap,itbi_vwap,krkn_vwap,lmax_vwap,okcn_vwap,stmp_vwap
timestamp,,,,,,,,,,
2021-02-24 00:00:00,48006.990396,47778.970482,47846.763010,47861.252787,47981.711507,47772.255562,47698.307100,47880.958914,47954.733900,47944.262770
2021-02-24 01:00:00,49607.923645,49441.592938,49196.187346,49343.265805,49458.079101,49509.699420,49179.026877,49250.834067,49422.816196,49220.299732
2021-02-24 02:00:00,50196.677303,50237.005036,50200.414314,50264.128120,50268.259968,50260.294897,50214.502350,50227.852912,50218.786802,50283.555918
2021-02-24 03:00:00,50948.000000,50608.443325,50523.291435,50528.684300,50494.398639,50533.802903,50537.044841,50458.462794,50435.243773,50472.637817
2021-02-24 04:00:00,50892.806229,50889.743300,50900.796481,50914.284328,50934.953286,51019.753640,50917.060833,50851.638004,51023.204299,50987.941708
2021-02-24 05:00:00,49727.934759,50344.152274,50383.987434,50537.358940,50528.602570,50416.281591,50421.092041,50473.959563,50399.457611,50440.124897
2021-02-24 06:00:00,49978.511632,49713.514206,50046.486487,50033.771901,49935.135891,49907.747367,49916.108053,50061.808328,49940.207735,49939.781897
2021-02-24 07:00:00,49680.100000,49899.138736,49861.581152,50032.963471,50034.858944,49944.963422,49994.271641,49996.280196,50030.400520,49974.235438
2021-02-24 08:00:00,50144.205464,50629.718533,50559.886413,50587.218865,50476.882959,50600.265267,50545.216768,50552.860398,50482.659430,50594.394449


In [163]:
vwap_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24 entries, 2021-02-24 00:00:00 to 2021-02-24 23:00:00
Freq: H
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   bfly_vwap  24 non-null     float64
 1   bfnx_vwap  24 non-null     float64
 2   bnus_vwap  24 non-null     float64
 3   btrx_vwap  24 non-null     float64
 4   gmni_vwap  24 non-null     float64
 5   itbi_vwap  24 non-null     float64
 6   krkn_vwap  24 non-null     float64
 7   lmax_vwap  24 non-null     float64
 8   okcn_vwap  24 non-null     float64
 9   stmp_vwap  24 non-null     float64
dtypes: float64(10)
memory usage: 2.1 KB


In [164]:
vwap_df.sum(axis=1)   #obtenir la somme de chaque ligne

timestamp
2021-02-24 00:00:00    478726.206427
2021-02-24 01:00:00    493629.725126
2021-02-24 02:00:00    502371.477620
2021-02-24 03:00:00    505540.009827
2021-02-24 04:00:00    509332.182108
2021-02-24 05:00:00    503672.951679
2021-02-24 06:00:00    499473.073498
2021-02-24 07:00:00    499448.793518
2021-02-24 08:00:00    505173.308546
2021-02-24 09:00:00    509307.308631
2021-02-24 10:00:00    505593.864135
2021-02-24 11:00:00    505013.950173
2021-02-24 12:00:00    509525.317456
2021-02-24 13:00:00    497963.245398
2021-02-24 14:00:00    492684.603509
2021-02-24 15:00:00    491700.192467
2021-02-24 16:00:00    497509.198666
2021-02-24 17:00:00    498109.369167
2021-02-24 18:00:00    497824.884049
2021-02-24 19:00:00    492863.162978
2021-02-24 20:00:00    489230.746760
2021-02-24 21:00:00    484850.161543
2021-02-24 22:00:00    485375.919025
2021-02-24 23:00:00    494791.557943
Freq: H, dtype: float64

In [165]:
def sum_rows(vwap_df):  #Calculer les valeurs totales pour chaque ligne et l'affecter à la nouvelle colonne avec le nom sum_rows
    vwap_df['sum_rows']=vwap_df.sum(axis=1)
    return vwap_df

In [166]:
sum_rows(vwap_df)

,bfly_vwap,bfnx_vwap,bnus_vwap,btrx_vwap,gmni_vwap,itbi_vwap,krkn_vwap,lmax_vwap,okcn_vwap,stmp_vwap,sum_rows
timestamp,,,,,,,,,,,
2021-02-24 00:00:00,48006.990396,47778.970482,47846.763010,47861.252787,47981.711507,47772.255562,47698.307100,47880.958914,47954.733900,47944.262770,478726.206427
2021-02-24 01:00:00,49607.923645,49441.592938,49196.187346,49343.265805,49458.079101,49509.699420,49179.026877,49250.834067,49422.816196,49220.299732,493629.725126
2021-02-24 02:00:00,50196.677303,50237.005036,50200.414314,50264.128120,50268.259968,50260.294897,50214.502350,50227.852912,50218.786802,50283.555918,502371.477620
2021-02-24 03:00:00,50948.000000,50608.443325,50523.291435,50528.684300,50494.398639,50533.802903,50537.044841,50458.462794,50435.243773,50472.637817,505540.009827
2021-02-24 04:00:00,50892.806229,50889.743300,50900.796481,50914.284328,50934.953286,51019.753640,50917.060833,50851.638004,51023.204299,50987.941708,509332.182108
2021-02-24 05:00:00,49727.934759,50344.152274,50383.987434,50537.358940,50528.602570,50416.281591,50421.092041,50473.959563,50399.457611,50440.124897,503672.951679
2021-02-24 06:00:00,49978.511632,49713.514206,50046.486487,50033.771901,49935.135891,49907.747367,49916.108053,50061.808328,49940.207735,49939.781897,499473.073498
2021-02-24 07:00:00,49680.100000,49899.138736,49861.581152,50032.963471,50034.858944,49944.963422,49994.271641,49996.280196,50030.400520,49974.235438,499448.793518
2021-02-24 08:00:00,50144.205464,50629.718533,50559.886413,50587.218865,50476.882959,50600.265267,50545.216768,50552.860398,50482.659430,50594.394449,505173.308546


In [167]:
def piece(vwap_df): #Création d'une ligne nommée pièce pour 10 échanges
    vwap_df['piece']=10
    return vwap_df

In [168]:
piece(vwap_df)

,bfly_vwap,bfnx_vwap,bnus_vwap,btrx_vwap,gmni_vwap,itbi_vwap,krkn_vwap,lmax_vwap,okcn_vwap,stmp_vwap,sum_rows,piece
timestamp,,,,,,,,,,,,
2021-02-24 00:00:00,48006.990396,47778.970482,47846.763010,47861.252787,47981.711507,47772.255562,47698.307100,47880.958914,47954.733900,47944.262770,478726.206427,10
2021-02-24 01:00:00,49607.923645,49441.592938,49196.187346,49343.265805,49458.079101,49509.699420,49179.026877,49250.834067,49422.816196,49220.299732,493629.725126,10
2021-02-24 02:00:00,50196.677303,50237.005036,50200.414314,50264.128120,50268.259968,50260.294897,50214.502350,50227.852912,50218.786802,50283.555918,502371.477620,10
2021-02-24 03:00:00,50948.000000,50608.443325,50523.291435,50528.684300,50494.398639,50533.802903,50537.044841,50458.462794,50435.243773,50472.637817,505540.009827,10
2021-02-24 04:00:00,50892.806229,50889.743300,50900.796481,50914.284328,50934.953286,51019.753640,50917.060833,50851.638004,51023.204299,50987.941708,509332.182108,10
2021-02-24 05:00:00,49727.934759,50344.152274,50383.987434,50537.358940,50528.602570,50416.281591,50421.092041,50473.959563,50399.457611,50440.124897,503672.951679,10
2021-02-24 06:00:00,49978.511632,49713.514206,50046.486487,50033.771901,49935.135891,49907.747367,49916.108053,50061.808328,49940.207735,49939.781897,499473.073498,10
2021-02-24 07:00:00,49680.100000,49899.138736,49861.581152,50032.963471,50034.858944,49944.963422,49994.271641,49996.280196,50030.400520,49974.235438,499448.793518,10
2021-02-24 08:00:00,50144.205464,50629.718533,50559.886413,50587.218865,50476.882959,50600.265267,50545.216768,50552.860398,50482.659430,50594.394449,505173.308546,10


In [169]:
def vwap_global(x):  #calcul vwap global
    x['vwap_global']=x['sum_rows']/x['piece']
    return x

In [170]:
vwap_global(vwap_df)

,bfly_vwap,bfnx_vwap,bnus_vwap,btrx_vwap,gmni_vwap,itbi_vwap,krkn_vwap,lmax_vwap,okcn_vwap,stmp_vwap,sum_rows,piece,vwap_global
timestamp,,,,,,,,,,,,,
2021-02-24 00:00:00,48006.990396,47778.970482,47846.763010,47861.252787,47981.711507,47772.255562,47698.307100,47880.958914,47954.733900,47944.262770,478726.206427,10,47872.620643
2021-02-24 01:00:00,49607.923645,49441.592938,49196.187346,49343.265805,49458.079101,49509.699420,49179.026877,49250.834067,49422.816196,49220.299732,493629.725126,10,49362.972513
2021-02-24 02:00:00,50196.677303,50237.005036,50200.414314,50264.128120,50268.259968,50260.294897,50214.502350,50227.852912,50218.786802,50283.555918,502371.477620,10,50237.147762
2021-02-24 03:00:00,50948.000000,50608.443325,50523.291435,50528.684300,50494.398639,50533.802903,50537.044841,50458.462794,50435.243773,50472.637817,505540.009827,10,50554.000983
2021-02-24 04:00:00,50892.806229,50889.743300,50900.796481,50914.284328,50934.953286,51019.753640,50917.060833,50851.638004,51023.204299,50987.941708,509332.182108,10,50933.218211
2021-02-24 05:00:00,49727.934759,50344.152274,50383.987434,50537.358940,50528.602570,50416.281591,50421.092041,50473.959563,50399.457611,50440.124897,503672.951679,10,50367.295168
2021-02-24 06:00:00,49978.511632,49713.514206,50046.486487,50033.771901,49935.135891,49907.747367,49916.108053,50061.808328,49940.207735,49939.781897,499473.073498,10,49947.307350
2021-02-24 07:00:00,49680.100000,49899.138736,49861.581152,50032.963471,50034.858944,49944.963422,49994.271641,49996.280196,50030.400520,49974.235438,499448.793518,10,49944.879352
2021-02-24 08:00:00,50144.205464,50629.718533,50559.886413,50587.218865,50476.882959,50600.265267,50545.216768,50552.860398,50482.659430,50594.394449,505173.308546,10,50517.330855


In [171]:
vwap_df.drop(vwap_df.columns[[10,11]], axis=1, inplace=True)

In [172]:
vwap_df

,bfly_vwap,bfnx_vwap,bnus_vwap,btrx_vwap,gmni_vwap,itbi_vwap,krkn_vwap,lmax_vwap,okcn_vwap,stmp_vwap,vwap_global
timestamp,,,,,,,,,,,
2021-02-24 00:00:00,48006.990396,47778.970482,47846.763010,47861.252787,47981.711507,47772.255562,47698.307100,47880.958914,47954.733900,47944.262770,47872.620643
2021-02-24 01:00:00,49607.923645,49441.592938,49196.187346,49343.265805,49458.079101,49509.699420,49179.026877,49250.834067,49422.816196,49220.299732,49362.972513
2021-02-24 02:00:00,50196.677303,50237.005036,50200.414314,50264.128120,50268.259968,50260.294897,50214.502350,50227.852912,50218.786802,50283.555918,50237.147762
2021-02-24 03:00:00,50948.000000,50608.443325,50523.291435,50528.684300,50494.398639,50533.802903,50537.044841,50458.462794,50435.243773,50472.637817,50554.000983
2021-02-24 04:00:00,50892.806229,50889.743300,50900.796481,50914.284328,50934.953286,51019.753640,50917.060833,50851.638004,51023.204299,50987.941708,50933.218211
2021-02-24 05:00:00,49727.934759,50344.152274,50383.987434,50537.358940,50528.602570,50416.281591,50421.092041,50473.959563,50399.457611,50440.124897,50367.295168
2021-02-24 06:00:00,49978.511632,49713.514206,50046.486487,50033.771901,49935.135891,49907.747367,49916.108053,50061.808328,49940.207735,49939.781897,49947.307350
2021-02-24 07:00:00,49680.100000,49899.138736,49861.581152,50032.963471,50034.858944,49944.963422,49994.271641,49996.280196,50030.400520,49974.235438,49944.879352
2021-02-24 08:00:00,50144.205464,50629.718533,50559.886413,50587.218865,50476.882959,50600.265267,50545.216768,50552.860398,50482.659430,50594.394449,50517.330855
